<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Software-and-conventions" data-toc-modified-id="Software-and-conventions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Software and conventions</a></span></li></ul></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Preparing-Inputs" data-toc-modified-id="Preparing-Inputs-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Preparing Inputs</a></span><ul class="toc-item"><li><span><a href="#Establishing-inputs" data-toc-modified-id="Establishing-inputs-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Establishing inputs</a></span></li></ul></li></ul></li><li><span><a href="#Compiling-with-spatial-soils" data-toc-modified-id="Compiling-with-spatial-soils-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compiling with spatial soils</a></span></li><li><span><a href="#Creating-the-observation-file" data-toc-modified-id="Creating-the-observation-file-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creating the observation file</a></span></li><li><span><a href="#The-calibration-workflow" data-toc-modified-id="The-calibration-workflow-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>The calibration workflow</a></span></li><li><span><a href="#Database-initialization" data-toc-modified-id="Database-initialization-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Database initialization</a></span></li><li><span><a href="#Domain-information" data-toc-modified-id="Domain-information-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Domain information</a></span></li><li><span><a href="#Experiment-initialization" data-toc-modified-id="Experiment-initialization-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Experiment initialization</a></span></li><li><span><a href="#Execution-of-calibration" data-toc-modified-id="Execution-of-calibration-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Execution of calibration</a></span></li></ul></div>

# Lesson - WRF-Hydro calibration

**NOTE:  This lesson is a demonstration only and is not currently supported**

## Overview

In this lesson, we will run a hands-on calibration experiment with the Croton watershed. Specifically, we will be calibrating the basin using observed and modeled streamflow at gage 1374581. 

### Software and conventions

To run the calibration workflow, there are multiple software requirements and pre-processing steps that need to take place prior to actual calibration. Currently, the calibration workflow relies on a set of Python and R libraries to execute various functions and plotting capabilities. This is in addition to existing requirements to compile and run WRF-Hydro. It's worth noting the current calibration workflow utilizes Python 2.7.X. In the future, migration to Python 3 is expected as Python 2 is phased out over 2019. The following Python libraries are used by the workflow:

1.) numpy
2.) NetCDF4
3.) pandas
4.) psutil

For R, there are several additional libraries required for proper analysis and plotting to take place. In the future, functions in R will be migrated over to Python to reduce dependencies. The following R libraries are needed for proper execution:

1.) data.table
2.) ggplot2
3.) ncdf4
4.) plyr
5.) boot
6.) sensitivity
7.) randtoolbox
8.) gridExtra

## Preprocessing

### Preparing Inputs

For each gage point being calibrated to, a set of domain, forcing, and observation files is needed for the workflow to run calibration on. In this step, we will be navigating to the directory containing necessary input files to run the Croton domain. However, a few steps need to be taken prior to establishing a calibration experiment. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/example_case/Gridded/DOMAIN
ls

#### Establishing inputs

In the step below, we are running several commands. The first is making symbolic links to the restart files from August 26th, 2011. The calibration workflow has a program to run spinups for you if you wish, but there is also an option to use spinup files for each model iteration if you provide them in the input file area. We will be doing that for this exercise to skip the spinup step. 

In addition, we will be making a symbolic link to the FORCING directory containing the input forcing files. We will also be creating an OBS subdirectory that will contain the input observations file for the gage we are calibrating too. Also, we will be copying over parameter input tables that are used by the model. Some of these may or may not be modified by the calibration workflow, depending on which parameters are being calibrated. 

Lastly, we will be renaming some of the files a bit as the calibration workflow has a set of expected file names for the input files (I.E. Fulldom.nc, wrfinput.nc, etc).

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/example_case/Gridded/DOMAIN
ln -s ../RESTART/HYDRO_RST.2011-08-26_00:00_DOMAIN1 HydroRestartSubstitute.nc
ln -s ../RESTART/RESTART.2011082600_DOMAIN1 LandRestartSubstitute.nc
ln -s hydro2dtbl.nc HYDRO_TBL_2D.nc
ln -s ../../FORCING
ln -s Fulldom_hires.nc Fulldom.nc
ln -s geo_em.d01.nc geo_em.nc
ln -s wrfinput_d01.nc wrfinput.nc
mkdir OBS
cp /home/docker/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Land_models/NoahMP/run/*.TBL .
cp /home/docker/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/HYDRO/CHANPARM.TBL .
cp /home/docker/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/HYDRO/HYDRO.TBL .
cp /home/docker/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Land_models/Noah/Run/URBPARM.TBL .
cp /home/docker/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Land_models/Noah/Run/VEGPARM.TBL .
ls -lh 

## Compiling with spatial soils

In the next step, we will be re-compiling the WRF-Hydro model code as was done earlier during the training. However, this time, we will be turning the SPATIAL_SOIL flag on. This is necessary for calibration purposes as the calibration workflow will be ajusting the distributed soil parameter NetCDF file that was created for this particular domain. We will also be turning the HYDRO_D flag off as this is unecessary during model iterations for the calibration workflow. 

In [ ]:
%%bash
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/; ./configure 2
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/setEnvar.sh \
~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/calibEnvars.sh
sed -i -e 's#export SPATIAL_SOIL=0#export SPATIAL_SOIL=1#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/calibEnvars.sh
sed -i -e 's#export HYDRO_D=1#export HYDRO_D=0#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/calibEnvars.sh

Run the compilation of the model code and ensure there are no errors as we did earlier in the training during earlier lessons. 

In [ ]:
%%bash
export PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/docker/miniconda2/bin:/home/docker/miniconda2/bin
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/; \
./compile_offline_NoahMP.sh calibEnvars.sh >> compile_calib.log 2>&1

In [ ]:
%%bash
tail -13 ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/compile_calib.log

## Creating the observation file

The calibration workflow currently expects an input R dataset file with the observations. The file should contain a data frame with columns that contain the observation gage ID value, date/time values in POSIXct format, along with the actual streamflow observation values. In the case below, we will be creating a very small R script that will read in the USGS csv file. The script will then ensure a data frame is created and saved to the OBS directory we created earlier. Note that the name of the data frame is "obsStrData" and the file name is "obsStrData.Rdata". This is very specific, and the calibration workflow looks for that specific filename during initialization. If the file is incorrectly named, errors will result.

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/example_case
echo "gStr <- '1374581'" >> convert_usgs_to_Rdata.R
echo "pathIn <- paste0('/home/docker/wrf-hydro-training/example_case/USGS_obs.csv')" >> convert_usgs_to_Rdata.R
echo "outPath <- paste0('/home/docker/wrf-hydro-training/example_case/Gridded/DOMAIN/OBS/obsStrData.Rdata')" >> convert_usgs_to_Rdata.R
echo "dataIn <- read.csv(pathIn)" >> convert_usgs_to_Rdata.R
echo "dataIn\$POSIXct <- as.POSIXct(dataIn\$dateTime,'%Y-%m-%d %H:%M:%S')" >> convert_usgs_to_Rdata.R
echo "dataIn\$obs <- dataIn\$streamflow_cms" >> convert_usgs_to_Rdata.R
echo "for (step in 1:length(dataIn\$site_no)){" >> convert_usgs_to_Rdata.R
echo "   dataIn\$site_no[step] <- toString(dataIn\$site_no[step])" >> convert_usgs_to_Rdata.R
echo "}" >> convert_usgs_to_Rdata.R
echo "dataIn <- subset(dataIn,site_no == gStr)" >> convert_usgs_to_Rdata.R
echo "obsStrData <- as.data.frame(dataIn)" >> convert_usgs_to_Rdata.R
echo "save(obsStrData,file=outPath)" >> convert_usgs_to_Rdata.R

In [ ]:
%%bash
Rscript /home/docker/wrf-hydro-training/example_case/convert_usgs_to_Rdata.R

## The calibration workflow

We will now navigate to where the calibration workflow code resides. At a top level, there are multiple programs that are used to run components of the calibration. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
ls

For our purposes, we will only be working with initDB.py, inputDomainMeta.py, jobInit.py, and calib.py. The initDB.py program initializes an sql-lite database file that contains the necessary tables to manage information about the workflow. The inputDomainMeta.py program will ingest information about our basin of interest, and place the data into the database tables. The jobInit.py program will ingest a configuration file we will create, place information about our experiment into tables, and create the necessary output directories where everything will be ran. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib/setup_files
ls

The setup files contain several useful files that are needed for proper execution of the workflow. The domainMetaTemplate.csv is a template file that user can fill in for each of their domains they are calibrating. You can put multiple domains within one csv file and enter the information all at once. For our exercise, we will be filliing out the template file for our Croton domain. This fille is then ingested into the workflow to store information in database tables. 

The calib_params.tbl lists the parameters for the model that can be modified. Some parameters are spatially distributed, while others are simply in a table file. The user can turn calibration for various parameters on/off by adjusting the calib_flag to either 1/0. There are also min/max/default values that specify the parameter space the workflow will be searching for, and adjusting parameter values. 

The setup.parm file is the main configuration file that helps guide the workflow to setup experiments, run them, and how to run them. We will be going over this in detail. 

The first step is to make copies of the files we need into our working space so we can modify them. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/example_case
mkdir Calib_HandsOn
cd /home/docker/wrf-hydro-training/example_case/Calib_HandsOn
cp /home/docker/wrf-hydro-training/PyWrfHydroCalib/setup_files/domainMetaTemplate.csv domainMeta_1374581.csv
cp /home/docker/wrf-hydro-training/PyWrfHydroCalib/setup_files/setup.parm setup_1374581.parm
cp /home/docker/wrf-hydro-training/PyWrfHydroCalib/setup_files/calib_params.tbl .

Below, we will be using the sed command to replace only what we need to replace for proper execution of the workflow. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/example_case/Calib_HandsOn
sed -i -e 's#/home/cuahsi/wrfHydroCalibration/04233300/DOMAIN#/home/docker/wrf-hydro-training/example_case/Gridded/DOMAIN#g' domainMeta_1374581.csv
sed -i -e 's#04233300#1374581#g' domainMeta_1374581.csv
sed -i -e 's#21983449#2#g' domainMeta_1374581.csv
sed -i -e 's#Six Mile Creek at Bethel Grove, NY#Croton#g' domainMeta_1374581.csv

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/example_case/Calib_HandsOn
sed -i -e 's#outDir = /home/cuahsi/calibOut#outDir = /home/docker/wrf-hydro-training/example_case/Calib_HandsOn#g' setup_1374581.parm
sed -i -e 's#jobName = TEST_CALIB#jobName = CUAHSI_TEST#g' setup_1374581.parm
sed -i -e 's#nCoresModel = 4#nCoresModel = 2#g' setup_1374581.parm
sed -i -e 's#runSens = 1#runSens = 0#g' setup_1374581.parm
sed -i -e 's#/home/cuahsi/PyWrfHydroCalibTest/setup_files/calib_params.tbl#/home/docker/wrf-hydro-training/example_case/Calib_HandsOn/calib_params.tbl#g' setup_1374581.parm
sed -i -e 's#optSpinFlag = 0#optSpinFlag = 1#g' setup_1374581.parm
sed -i -e 's#numIter = 3#numIter = 10#g' setup_1374581.parm
sed -i -e 's#objectiveFunction = Nse#objectiveFunction = Kge#g' setup_1374581.parm
sed -i -e 's#email = john.doe@youremail.com#email = karsten@ucar.edu#g' setup_1374581.parm
sed -i -e 's#/home/cuahsi/wrfHydroCalibration/wrf_hydro_model/trunk/NDHMS/Run/wrf_hydro.exe#/home/docker/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe#g' setup_1374581.parm
sed -i -e 's#/home/cuahsi/wrfHydroCalibration/04233300#/home/docker/wrf-hydro-training/example_case/Gridded/DOMAIN#g' setup_1374581.parm
sed -i -e 's#bCalibDate = 2013-07-01#bCalibDate = 2011-08-26#g' setup_1374581.parm
sed -i -e 's#eCalibDate = 2013-08-01#eCalibDate = 2011-08-30#g' setup_1374581.parm
sed -i -e 's#bCalibEvalDate = 2013-07-05#bCalibEvalDate = 2011-08-27#g' setup_1374581.parm
sed -i -e 's#dtChSec = 300#dtChSec = 10#g' setup_1374581.parm
sed -i -e 's#chanRtOpt = 2#chanRtOpt = 3#g' setup_1374581.parm
sed -i -e 's#udmpOpt = 1#udmpOpt = 0#g' setup_1374581.parm

cat setup_1374581.parm

Below we are turning off the hvt and mfsno parameters for calibrtion, and turning on Bw/MannN, which are channel parameters in CHANPARM.TBL.

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/example_case/Calib_HandsOn
sed -i -e 's#hvt,         1#hvt,         0#g' calib_params.tbl
sed -i -e 's#mfsno,       1#mfsno,       0#g' calib_params.tbl
sed -i -e 's#Bw,          0#Bw,          1#g' calib_params.tbl
sed -i -e 's#MannN,       0#MannN,       1#g' calib_params.tbl

## Database initialization

The first step is to initialize the database file, which is simply a sqlite DB file. 

*IMPORTANT*: This file will contain all data on your workflow, including status values, parameter values, and error statistics. Be very careful when moving the file as once it's deleted, you cannot replicate it, unless you re-run the calibration experiment. 

The database file contains several tables. Some tables have metadata on the workflow, including paths to input files, domain information for each gage being calibrated. Other tables contain actual parameter values for each iteration during the calibration, along with the associated statistics. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python initDB.py
ls

## Domain information

This next step will utilize the csv file we modified earlier containing information for the basin of interest, along with the inputDomainMeta.py program to ingest domain information into the database tables. The program will ensure all necessary input files are available, including the observations file.

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python inputDomainMeta.py --help

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python inputDomainMeta.py /home/docker/wrf-hydro-training/example_case/Calib_HandsOn/domainMeta_1374581.csv

In Addition to top-level programs to execute the workflow components, there are utility programs for use to either print information to the screen for the user, or extract information about a calibration to an output file. The example below prints information about our domain we are calibrating to the screen for the user to verify it looks correct. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python util/printDomainInfo.py wrfHydroCalib.db

## Experiment initialization

Below, we run the jobInit.py program to initialize our calibration experiment, and set everything up. It's worth noting this step does not actually initialize any model simulations, etc, but simply enters information into the database tables on what we are doing, along with creating all the output directories where results of our calibration will reside. 

Once the initialization has successfully completed, a message is returned that provides the user with a unique identification number for this particular experiment. Note that the user can include multiple domains across a region into one calibration experiment. This is specified in the configuration file through the SQL command. 

The ID that is returned is then used as an argument to remaining calibration programs to execute components (spinup,calibration,sensitivity analysis,validation).

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python jobInit.py --help

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python jobInit.py /home/docker/wrf-hydro-training/example_case/Calib_HandsOn/setup_1374581.parm

Once again, we will be running one of the optional utility programs to ensure the details of our experiment look good. Note that we are passing the ID value returned to us in the previous step. It's possible the user may have multiple calibration experiments in one database file. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python util/printJobMeta.py wrfHydroCalib.db 1

## Execution of calibration

Fortunately, the actual execution of the calibration is fairly simple. The only required argument is the ID number for our experiment. The workflow then handles the rest from there. Once the calibration is complete, a complete message is communicated back to the user. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python calib.py --help

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib
python calib.py 1

© UCAR 2018